**Group-08**<br/>
<font style="color:red"> **Belhassen Ghoul <br/> Robin Ehrensperger <br/> Dominic Diedenhofen**</font>

In [12]:
import torch
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor
from torchsummary import summary
import numpy as np
import matplotlib.pyplot as plt
import torchvision
import sys

from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter("runs/mnist")

### Loading Data

In [13]:
training_data = datasets.mnist.FashionMNIST(root="data", train=True, download=True, transform=ToTensor())
test_data = datasets.mnist.FashionMNIST(root="data", train=False, download=True, transform=ToTensor())

In [14]:
training_data, validation_data = torch.utils.data.random_split(training_data, [50000, 10000])

In [15]:
print(len(training_data),len(validation_data),len(test_data))

50000 10000 10000


### MLP


In [16]:
def mlp(units = [28*28,250,80,10]):
    seq = [torch.nn.Flatten()]
    for i in range(len(units)-2):
        seq.append(torch.nn.Linear(units[i],units[i+1]))
        seq.append(torch.nn.Sigmoid())
    seq.append(torch.nn.Linear(units[-2],units[-1]))
    return torch.nn.Sequential(*seq)

In [17]:
model = mlp()
from torchsummary import summary
summary(model, (1,28,28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
           Flatten-1                  [-1, 784]               0
            Linear-2                  [-1, 250]         196,250
           Sigmoid-3                  [-1, 250]               0
            Linear-4                   [-1, 80]          20,080
           Sigmoid-5                   [-1, 80]               0
            Linear-6                   [-1, 10]             810
Total params: 217,140
Trainable params: 217,140
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.01
Params size (MB): 0.83
Estimated Total Size (MB): 0.84
----------------------------------------------------------------


### Training

Implement the training / evaluation loop

Remember training / validation cost and accuracy per epoch. 

In [18]:
def train_eval(model, optimizer, nepochs, training_loader, test_loader, scheduler=None):
    cost_hist = []
    cost_hist_test = []
    acc_hist = []
    acc_hist_test = []

    cost_ce = torch.nn.CrossEntropyLoss()

    runningLoss = 0.0 
    runningCorrect = 0.0
    for epoch in range(nepochs):
        model.train()
        size = len(training_loader.dataset)
        nbatches = len(training_loader)
        size_test = len(test_loader.dataset)
        nbatches_test = len(test_loader)
        cost, acc = 0.0, 0.0
        for batch, (X, Y) in enumerate(training_loader):
            pred = model(X)
            loss = cost_ce(pred, Y)
            cost += loss.item()
            acc += (pred.argmax(dim=1) == Y).type(torch.float).sum().item()

            # gradient, parameter update
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            runningLoss += loss.item()
            runningCorrect += (pred.argmax(dim=1) == Y).type(torch.float).sum().item()
            writer.add_scalar("training loss", runningLoss, epoch*nbatches+batch)
            writer.add_scalar("accuracy loss", runningCorrect, epoch*nbatches+batch)
            runningLoss = 0.0
            runningCorrect = 0.0

            writer.add_graph(model, X)
            writer.close()
        if scheduler:
            scheduler.step()
        cost /= nbatches
        acc /= size
    
        model.eval()
        cost_test, acc_test = 0.0, 0.0        
        with torch.no_grad():
            for X, Y in test_loader:
                pred = model(X)
                cost_test += cost_ce(pred, Y).item()
                acc_test += (pred.argmax(dim=1) == Y).type(torch.float).sum().item()
        cost_test /= nbatches_test
        acc_test /= size_test

        print("Epoch %i: %f, %f, %f, %f"%(epoch, cost, acc, cost_test, acc_test))
        cost_hist.append(cost)
        cost_hist_test.append(cost_test)
        acc_hist.append(acc)
        acc_hist_test.append(acc_test)
    return cost_hist, cost_hist_test, acc_hist, acc_hist_test

In [19]:
nbatch = 64
nepochs =20

training_loader = DataLoader(training_data, batch_size=nbatch, shuffle=True)
validation_loader = DataLoader(validation_data, batch_size=nbatch, shuffle=True)

model = mlp()
optimizer = torch.optim.SGD(model.parameters(),lr=0.8)

cost_hist, cost_hist_test, acc_hist, acc_hist_test = train_eval(model, optimizer, nepochs, training_loader, validation_loader)



Epoch 0: 0.899447, 0.650440, 0.596000, 0.792300
Epoch 1: 0.514793, 0.810700, 0.597713, 0.778100
Epoch 2: 0.452955, 0.834280, 0.447927, 0.839300
Epoch 3: 0.415104, 0.847440, 0.459456, 0.835700
Epoch 4: 0.387457, 0.859180, 0.387945, 0.857200
Epoch 5: 0.368129, 0.863140, 0.572586, 0.777200
Epoch 6: 0.353074, 0.869180, 0.573192, 0.781900
Epoch 7: 0.339449, 0.874460, 0.332906, 0.873000
Epoch 8: 0.326345, 0.878520, 0.341865, 0.872200
Epoch 9: 0.316329, 0.883380, 0.879270, 0.690100
Epoch 10: 0.308295, 0.885420, 0.471664, 0.838200
Epoch 11: 0.301618, 0.887040, 0.340638, 0.875700
Epoch 12: 0.291519, 0.891540, 0.375130, 0.863500
Epoch 13: 0.285340, 0.893400, 0.427403, 0.851800
Epoch 14: 0.279000, 0.895840, 0.350703, 0.863600
Epoch 15: 0.273223, 0.897700, 0.362765, 0.864400
Epoch 16: 0.265776, 0.900200, 0.301215, 0.890400
Epoch 17: 0.259582, 0.902040, 0.344069, 0.873100
Epoch 18: 0.254696, 0.903960, 0.385500, 0.857800
Epoch 19: 0.249713, 0.906420, 0.292794, 0.893500


### Plots and Comments (for the different steps described above) 

In [20]:
examples = iter(training_loader)
example_data, example_labels = examples.next()

img_grid = torchvision.utils.make_grid(example_data)
writer.add_image("fashon",img_grid)
writer.close()